In [1]:
from PIL import Image
import pandas as pd
from pyecharts.charts import *
from pyecharts.components import Table
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
import random
import datetime
from impala.dbapi import connect
from impala.util import as_pandas

In [2]:
# 得到连接，
conn = connect(host='121.41.31.137', port=10000, auth_mechanism='PLAIN', user='dingran')
# 得到句柄
cursor = conn.cursor()

In [3]:
dt=str(datetime.date.today()+datetime.timedelta(-1))

In [4]:
dt='2022-06-05'
dt

'2022-06-05'

In [5]:
str(datetime.datetime.now())

'2022-06-07 13:49:15.366722'

## 定义需要归因的埋点

### 资源位(待归因事件)

#### 一级待归因事件

In [6]:
attribution_level_one_events="""
select
id
,dt
,event_code
,event_type
,os_type
,get_json_object(properties,'$.page_id') `page_id`
,get_json_object(properties,'$.page_title') `page_title`
,get_json_object(properties,'$.floor_name') `floor_name`
,get_json_object(properties,'$.item_name') `item_name`
,get_json_object(properties,'$.goods_id') `goods_code`
,get_json_object(properties,'$.goods_source') `goods_source`
,nvl(nvl(get_json_object(properties,'$.parameter_transmit'),get_json_object(properties,'$.is_recommend')),'-1') `is_recommend` -- 是否推荐，1是、其他不是
,event_time
,user_id
-- 增加订单统计的维度
,'' order_code
,'' deal_with_amount
,'' order_status
,'' risk_audit_ststus
,'' order_date
,'' pay_date
,properties
from ods_xxyp_event.ods_app_event_data_detail_dt
where dt="{dt}" 
and app_name='小象优品'
-- 一级资源位
and (
	-- 广告位：APP启动页广告点击
	(event_code='theBigPicOfAppStartingClick')
	-- 广告位：首页的插屏点击
	or
	(event_code='theSuperDynamicInterstitialWindowsClick')
	-- 首页的点击
	or
	(get_json_object(properties,'$.page_id')='1'
	and event_type='ck'
	)
	-- 个人中心页的点击
	or
	(get_json_object(properties,'$.page_id')='20new_personal'
	and event_type='ck'
	)
	-- 钱包页资源位点击
	or
	(get_json_object(properties,'$.page_id')='qianbao_home'
	and event_type='ck'
	)
	-- plus中心页点击
	or
	(get_json_object(properties,'$.page_id')='plus_center'
	and event_type='ck'
	)
)
and user_id='24997376'
""".format(dt=dt)

#### 二级待归因事件

In [7]:
attribution_level_two_events='''
select
id
,dt
,event_code
,event_type
,os_type
,get_json_object(properties,'$.page_id') `page_id`
,get_json_object(properties,'$.page_title') `page_title`
,get_json_object(properties,'$.floor_name') `floor_name`
,get_json_object(properties,'$.item_name') `item_name`
,get_json_object(properties,'$.goods_id') `goods_code`
,get_json_object(properties,'$.goods_source') `goods_source`
,nvl(nvl(get_json_object(properties,'$.parameter_transmit'),get_json_object(properties,'$.is_recommend')),'-1') `is_recommend` -- 是否推荐，1是、其他不是
,event_time
,user_id
-- 增加订单统计的维度
,'' order_code
,'' deal_with_amount
,'' order_status
,'' risk_audit_ststus
,'' order_date
,'' pay_date
,properties
from ods_xxyp_event.ods_app_event_data_detail_dt
where dt="{dt}" 
and app_name='小象优品'
-- 二级资源位
and (
	-- 其他原生专场的点击
	(get_json_object(properties,'$.page_id') in ("2","3","4","5","6","7","8","9","10","11","12","13","14","15")
	and event_type='ck'
	)
	-- 分类页的点击(category_product_click商品点击事件不是类目页的，page_id和类目页的一样，没page_title)
	or
	(get_json_object(properties,'$.page_id')='category_page'
	and event_type='ck'
    and get_json_object(properties,'$.page_title')='分类页' -- 限制下，只要分类页的，不然会对其他页面造成截断
	)
	-- 分类页右边坑位的点击，埋点event_code目前是sw
	or
	(event_code='category_page_right_item_ck')

	-- 广告位：所有二级页广告点击
	or
	(event_code in ("account_benefits","plusHome_middleBanner_ck","chargeTelephoneFees_TopAd_ck","chargeTelephoneFees_BottomAd_ck","wallet_bottomBox_ck","loanRecordPage_Banner_ck","repaymentPage_interstitialAd_ck","repaymentPage_banner_ck","OrderRatePage_banner_ck","OrderRatePage_banner_ck")
	and event_type='ck'
	)
	-- 所有未来添加的广告位
	or 
	(get_json_object(properties,'$.is_ad_sys')='1')
	-- 搜索页的点击
	or
	(get_json_object(properties,'$.page_id')='searchpage'
	and event_type='ck'
	)
    -- 搜索结果也的所有点击
    or
    (get_json_object(properties,'$.page_id')='goods_search_result'
	and event_type='ck'
	)
    
)
and user_id='24997376'
'''.format(dt=dt)

In [8]:
attribution_level_two_events

'\nselect\nid\n,dt\n,event_code\n,event_type\n,os_type\n,get_json_object(properties,\'$.page_id\') `page_id`\n,get_json_object(properties,\'$.page_title\') `page_title`\n,get_json_object(properties,\'$.floor_name\') `floor_name`\n,get_json_object(properties,\'$.item_name\') `item_name`\n,get_json_object(properties,\'$.goods_id\') `goods_code`\n,get_json_object(properties,\'$.goods_source\') `goods_source`\n,nvl(nvl(get_json_object(properties,\'$.parameter_transmit\'),get_json_object(properties,\'$.is_recommend\')),\'-1\') `is_recommend` -- 是否推荐，1是、其他不是\n,event_time\n,user_id\n-- 增加订单统计的维度\n,\'\' order_code\n,\'\' deal_with_amount\n,\'\' order_status\n,\'\' risk_audit_ststus\n,\'\' order_date\n,\'\' pay_date\n,properties\nfrom ods_xxyp_event.ods_app_event_data_detail_dt\nwhere dt="2022-06-05" \nand app_name=\'小象优品\'\n-- 二级资源位\nand (\n\t-- 其他原生专场的点击\n\t(get_json_object(properties,\'$.page_id\') in ("2","3","4","5","6","7","8","9","10","11","12","13","14","15")\n\tand event_type=\'ck\'\n\

#### 三级待归因事件

In [9]:
attribution_level_three_events='''
select
id
,dt
,event_code
,event_type
,os_type
,get_json_object(properties,'$.page_id') `page_id`
,get_json_object(properties,'$.page_title') `page_title`
,get_json_object(properties,'$.floor_name') `floor_name`
,get_json_object(properties,'$.item_name') `item_name`
,get_json_object(properties,'$.goods_id') `goods_code`
,get_json_object(properties,'$.goods_source') `goods_source`
,nvl(nvl(get_json_object(properties,'$.is_recommend'),get_json_object(properties,'$.parameter_transmit')),'-1') `is_recommend` -- 是否推荐，1是、其他不是
,event_time
,user_id
-- 增加订单统计的维度
,'' order_code
,'' deal_with_amount
,'' order_status
,'' risk_audit_ststus
,'' order_date
,'' pay_date
,properties
from ods_xxyp_event.ods_app_event_data_detail_dt
where dt="{dt}" 
and app_name='小象优品'
-- 二级资源位
and event_type='ck'
and (get_json_object(properties,'$.goods_id') is not null or get_json_object(properties,'$.good_id') is not null)
and event_code!='' -- 排除掉事件名为空的(颜值卡、会员中心页的商品点击event_code为空的)

-- 排除直接加购的事件
and event_code not in ("micro_goods_detail_goods_ck","micro_goods_detail_button_ck","micro_goods_detail_specification_window_ck","use_coupon_result_toshopcart_ck","use_coupon_goods_detail_specification_window_ck","add_to_shopping_cart_now","interest_free_page_toshopcart_ck","interest_free_page_goods_detail_window_ck","nOptional_page_toshopcart_ck","nOptional_page_goods_detail_window_ck","nOptional_page_tobuy_ck","reduction_page_toshopcart_ck","reduction_page_goods_detail_window_ck","reduction_page_tobuy_ck","discount_page_toshopcart_ck","discount_page_goods_detail_window_ck","discount_page_tobuy_ck","brand_sale_page_toshopcart_ck","brand_sale_page_goods_detail_window_ck","goods_brand_goods_search_toshopcart_ck","brand_goods_search_goods_detail_window_ck")
-- 程序中商品详情页的和购物车页的除了商品推荐的，其他的都排除掉
-- and get_json_object(properties,'$.page_id')!='goods_detail'

and user_id='24997376'
'''.format(dt=dt)

### 商品详情页(前项关联事件)

In [10]:
relation_events="""
select
id
,dt
,event_code
,event_type
,os_type
,get_json_object(properties,'$.page_id') `page_id`
,get_json_object(properties,'$.page_title') `page_title`
,get_json_object(properties,'$.floor_name') `floor_name`
,get_json_object(properties,'$.item_name') `item_name`
,get_json_object(properties,'$.goods_id') `goods_code`
,get_json_object(properties,'$.goods_source') `goods_source`
,'-1' `is_recommend`
,event_time
,user_id
-- 增加订单统计的维度
,'' order_code
,'' deal_with_amount
,'' order_status
,'' risk_audit_ststus
,'' order_date
,'' pay_date
,'' `properties`
from ods_xxyp_event.ods_app_event_data_detail_dt
where dt="{dt}" 
and app_name='小象优品'
-- 商品详情页(前向关联事件)
and(
-- 实物详情页
(
event_code='page_start'
and event_type='sf'
and get_json_object(properties,'$.page_id')='goods_detail'
)
-- 虚拟详情页
or
(
event_code='page_start'
and event_type='sf'
and get_json_object(properties,'$.page_id')='virtual_goods_detail'
)

-- 微详情页或者其他页面不用访问商品详情页的可以直接购买或加购的
or
(
event_code in ("micro_goods_detail_button_ck","micro_goods_detail_specification_window_ck","use_coupon_result_toshopcart_ck","use_coupon_goods_detail_specification_window_ck","add_to_shopping_cart_now","interest_free_page_toshopcart_ck","interest_free_page_goods_detail_window_ck","nOptional_page_toshopcart_ck","nOptional_page_goods_detail_window_ck","nOptional_page_tobuy_ck","reduction_page_toshopcart_ck","reduction_page_goods_detail_window_ck","reduction_page_tobuy_ck","discount_page_toshopcart_ck","discount_page_goods_detail_window_ck","discount_page_tobuy_ck","brand_sale_page_toshopcart_ck","brand_sale_page_goods_detail_window_ck","goods_brand_goods_search_toshopcart_ck","brand_goods_search_goods_detail_window_ck")
)
)

and user_id='24997376'
""".format(dt=dt)

### 订单数据(目标转化事件)

In [11]:
target_events="""
select
id
,order_date `dt`
,'create_order' event_code
,'uc' event_type
,'' os_type
,'' `page_id`
,'' `page_title`
,'' `floor_name`
,'' `item_name`
,goods_code
,good_source `goods_source`
,'-1' `is_recommend`
,unix_timestamp(order_datetime)*1000 event_time -- 转换成毫秒
,user_id
,order_code
,deal_with_amount
,order_status
,risk_audit_ststus
,order_date
,pay_date
,'' `properties`
from dwd_xxyp_order.fact_dwd_xxyp_order_main_order_d_a_dt
where order_date="{dt}"
and order_source<6
and order_type=2

and user_id='24997376'
""".format(dt=dt)

## 查询明细数据

### 资源位明细数据

#### 一级待归因数据

In [12]:
# 执行查询
cursor.execute(attribution_level_one_events)
# 获取列名
columns = [col[0] for col in cursor.description]

In [13]:
# 查询结果
attribution_level_one_events_data=cursor.fetchall()

In [14]:
attribution_level_one_events_df=pd.DataFrame(attribution_level_one_events_data,columns=columns)

In [15]:
attribution_level_one_events_df['type']='attribution'
attribution_level_one_events_df['attribution_level']='1'

In [16]:
attribution_level_one_events_df.head()

,id,dt,event_code,event_type,os_type,page_id,page_title,floor_name,item_name,goods_code,...,user_id,order_code,deal_with_amount,order_status,risk_audit_ststus,order_date,pay_date,properties,type,attribution_level
0,585816279819489281,2022-06-05,20new_personal_ck,ck,android,20new_personal,个人中心,我的订单,我的订单,None,...,24997376,,,,,,,"{""floor_name"":""我的订单"",""page_id"":""20new_personal...",attribution,1
1,585887902622814220,2022-06-05,20new_personal_ck,ck,android,20new_personal,个人中心,个人设置,账户设置,None,...,24997376,,,,,,,"{""floor_name"":""个人设置"",""page_id"":""20new_personal...",attribution,1
2,585888388486533121,2022-06-05,recommend_item_ck,ck,android,1,小象首页,商品tab,底部tab（新）,201703239,...,24997376,,,,,,,"{""goods_name"":""Apple iPad Pro 12.9英寸平板电脑 2021年...",attribution,1
3,585888744897781763,2022-06-05,recommend_item_ck,ck,android,1,小象首页,商品tab,底部tab（新）,201936364,...,24997376,,,,,,,"{""goods_name"":""【二手苹果Apple】iPhone 12mini 国行/国际版...",attribution,1
4,585888895661903875,2022-06-05,recommend_item_ck,ck,android,1,小象首页,商品tab,底部tab（新）,201883060,...,24997376,,,,,,,"{""goods_name"":""Apple iPad mini 8.3英寸 平板电脑 2021...",attribution,1


#### 二级待归因数据

In [17]:
# 执行查询
cursor.execute(attribution_level_two_events)
# 获取列名
columns = [col[0] for col in cursor.description]

In [18]:
# 查询结果
attribution_level_two_events_data=cursor.fetchall()

In [19]:
attribution_level_two_events_df=pd.DataFrame(attribution_level_two_events_data,columns=columns)

In [20]:
attribution_level_two_events_df['type']='attribution'
attribution_level_two_events_df['attribution_level']='2'

In [21]:
attribution_level_two_events_df.head()

,id,dt,event_code,event_type,os_type,page_id,page_title,floor_name,item_name,goods_code,...,user_id,order_code,deal_with_amount,order_status,risk_audit_ststus,order_date,pay_date,properties,type,attribution_level


#### 三级待归因数据

In [22]:
# 执行查询
cursor.execute(attribution_level_three_events)
# 获取列名
columns = [col[0] for col in cursor.description]

In [23]:
# 查询结果
attribution_level_three_events_data=cursor.fetchall()

In [24]:
attribution_level_three_events_df=pd.DataFrame(attribution_level_three_events_data,columns=columns)

In [25]:
attribution_level_three_events_df['type']='attribution'
attribution_level_three_events_df['attribution_level']='3'

In [26]:
attribution_level_three_events_df.head()

,id,dt,event_code,event_type,os_type,page_id,page_title,floor_name,item_name,goods_code,...,user_id,order_code,deal_with_amount,order_status,risk_audit_ststus,order_date,pay_date,properties,type,attribution_level
0,585888388486533121,2022-06-05,recommend_item_ck,ck,android,1,小象首页,商品tab,底部tab（新）,201703239,...,24997376,,,,,,,"{""goods_name"":""Apple iPad Pro 12.9英寸平板电脑 2021年...",attribution,3
1,585888744897781763,2022-06-05,recommend_item_ck,ck,android,1,小象首页,商品tab,底部tab（新）,201936364,...,24997376,,,,,,,"{""goods_name"":""【二手苹果Apple】iPhone 12mini 国行/国际版...",attribution,3
2,585888895661903875,2022-06-05,recommend_item_ck,ck,android,1,小象首页,商品tab,底部tab（新）,201883060,...,24997376,,,,,,,"{""goods_name"":""Apple iPad mini 8.3英寸 平板电脑 2021...",attribution,3
3,585888895687069701,2022-06-05,goods_detail_ck,ck,android,goods_detail,商品详情页,None,None,201867657,...,24997376,,,,,,,"{""click_type"":""tobuy_ck"",""page_id"":""goods_deta...",attribution,3
4,585888970001485824,2022-06-05,goods_detail_specification_window_ck,ck,android,goods_detail,商品详情页,None,None,201867657,...,24997376,,,,,,,"{""goods_name"":""Apple iPad Pro 11英寸 平板电脑 2021年新...",attribution,3


##### 把商品详情页和购物车页非推荐商品的点击给过滤掉

In [27]:
attribution_level_three_events_df.head(15)

,id,dt,event_code,event_type,os_type,page_id,page_title,floor_name,item_name,goods_code,...,user_id,order_code,deal_with_amount,order_status,risk_audit_ststus,order_date,pay_date,properties,type,attribution_level
0,585888388486533121,2022-06-05,recommend_item_ck,ck,android,1,小象首页,商品tab,底部tab（新）,201703239,...,24997376,,,,,,,"{""goods_name"":""Apple iPad Pro 12.9英寸平板电脑 2021年...",attribution,3
1,585888744897781763,2022-06-05,recommend_item_ck,ck,android,1,小象首页,商品tab,底部tab（新）,201936364,...,24997376,,,,,,,"{""goods_name"":""【二手苹果Apple】iPhone 12mini 国行/国际版...",attribution,3
2,585888895661903875,2022-06-05,recommend_item_ck,ck,android,1,小象首页,商品tab,底部tab（新）,201883060,...,24997376,,,,,,,"{""goods_name"":""Apple iPad mini 8.3英寸 平板电脑 2021...",attribution,3
3,585888895687069701,2022-06-05,goods_detail_ck,ck,android,goods_detail,商品详情页,None,None,201867657,...,24997376,,,,,,,"{""click_type"":""tobuy_ck"",""page_id"":""goods_deta...",attribution,3
4,585888970001485824,2022-06-05,goods_detail_specification_window_ck,ck,android,goods_detail,商品详情页,None,None,201867657,...,24997376,,,,,,,"{""goods_name"":""Apple iPad Pro 11英寸 平板电脑 2021年新...",attribution,3


In [28]:
# 删除商品详情页非商品推荐的点击
attribution_level_three_events_df.drop(attribution_level_three_events_df[(attribution_level_three_events_df['page_id']=='goods_detail') & (attribution_level_three_events_df['is_recommend']!='1')].index, inplace=True)

In [29]:
# 删除购物车页非商品推荐的点击
attribution_level_three_events_df.drop(attribution_level_three_events_df[(attribution_level_three_events_df['page_id']=='shopping_cart') & (attribution_level_three_events_df['is_recommend']!='1')].index, inplace=True)

In [30]:
attribution_level_three_events_df[attribution_level_three_events_df['page_id'].isnull()].head()

,id,dt,event_code,event_type,os_type,page_id,page_title,floor_name,item_name,goods_code,...,user_id,order_code,deal_with_amount,order_status,risk_audit_ststus,order_date,pay_date,properties,type,attribution_level


In [31]:
# 删除没有page_id的 add_to_shopping_cart_now 事件
attribution_level_three_events_df.dropna(subset=['page_id'],inplace=True)

In [32]:
attribution_level_three_events_df[attribution_level_three_events_df['page_id'].isnull()].head()

,id,dt,event_code,event_type,os_type,page_id,page_title,floor_name,item_name,goods_code,...,user_id,order_code,deal_with_amount,order_status,risk_audit_ststus,order_date,pay_date,properties,type,attribution_level


#### 合并待归因的数据

In [33]:
attribution_df=pd.concat([attribution_level_one_events_df,attribution_level_two_events_df,attribution_level_three_events_df])

In [34]:
attribution_df.to_csv('归因-待归因事件{dt}.csv'.format(dt=dt), index=False)

### 商品详情页明细数据

In [35]:
# 执行查询
cursor.execute(relation_events)
# 获取列名
columns = [col[0] for col in cursor.description]

In [36]:
# 查询结果
relation_data=cursor.fetchall()

In [37]:
relation_df=pd.DataFrame(relation_data,columns=columns)

In [38]:
relation_df['type']='relation'
# 设置 100 ,为了同时属于前项关联时间和三级待归因事件的排序
relation_df['attribution_level']='9999'

In [39]:
relation_df.head()

,id,dt,event_code,event_type,os_type,page_id,page_title,floor_name,item_name,goods_code,...,user_id,order_code,deal_with_amount,order_status,risk_audit_ststus,order_date,pay_date,properties,type,attribution_level
0,585816217508515840,2022-06-05,page_start,sf,android,goods_detail,商品详情页,None,None,201701171,...,24997376,,,,,,,,relation,9999
1,585888762014732288,2022-06-05,micro_goods_detail_button_ck,ck,android,micro_goods_detail,微详情页,商品微详情,立即购买,201936364,...,24997376,,,,,,,,relation,9999
2,585888831333867520,2022-06-05,micro_goods_detail_specification_window_ck,ck,android,micro_goods_detail,微详情页,商品微详情,None,201936364,...,24997376,,,,,,,,relation,9999
3,585888895687069696,2022-06-05,page_start,sf,android,goods_detail,商品详情页,None,None,201867657,...,24997376,,,,,,,,relation,9999
4,585887699652190208,2022-06-05,page_start,sf,android,goods_detail,商品详情页,None,None,201895126,...,24997376,,,,,,,,relation,9999


In [40]:
relation_df.to_csv('归因-商品详情页{dt}.csv'.format(dt=dt), index=False)

### 订单明细数据

In [41]:
# 执行查询
cursor.execute(target_events)
# 获取列名
columns = [col[0] for col in cursor.description]

In [42]:
# 查询结果
target_data=cursor.fetchall()

In [43]:
target_df=pd.DataFrame(target_data,columns=columns)

In [44]:
target_df['type']='target'
target_df['attribution_level']='0'

In [45]:
target_df.to_csv('归因-商品订单{dt}.csv'.format(dt=dt), index=False)

### 合并所有数据明细

In [46]:
all_df=pd.concat([attribution_df,relation_df,target_df])

In [47]:
# 临时保存用户明细
user_info=all_df.query('["23948389"] in user_id').sort_values(by=['event_time','attribution_level'],ascending=False)
# user_info.to_excel('用户明细x{dt}.xlsx')

In [48]:
user_info.query("type=='target'").head()

,id,dt,event_code,event_type,os_type,page_id,page_title,floor_name,item_name,goods_code,...,user_id,order_code,deal_with_amount,order_status,risk_audit_ststus,order_date,pay_date,properties,type,attribution_level


In [49]:
all_df.query("type=='target'").head()

,id,dt,event_code,event_type,os_type,page_id,page_title,floor_name,item_name,goods_code,...,user_id,order_code,deal_with_amount,order_status,risk_audit_ststus,order_date,pay_date,properties,type,attribution_level
0,155366442,2022-06-05,create_order,uc,,,,,,201936364,...,24997376,6913593515484517089288,259900,22,0,2022-06-05,None,,target,0
1,155366464,2022-06-05,create_order,uc,,,,,,201867657,...,24997376,1913593517314030632961,752900,22,0,2022-06-05,None,,target,0


In [50]:
# 备份下数据
clear_data=all_df.copy()

In [51]:
# 转换成字符串类型
clear_data['user_id']=clear_data['user_id'].astype('str')
clear_data['event_time']=clear_data['event_time'].astype('str')

## 开始洗结果数据

In [52]:
# user_list=["25828203","13519932"]

In [53]:
user_list=list(clear_data['user_id'].unique())

### 一次把所有级别的待归因都找出来

In [54]:
# 存放所有查找到的资源位
find_all_df=None
for user_id in user_list:
    # 取该用户的所有数据,并按照时间和归因等级降序，这样即使不同级别的归因事件是一样的，也都能归到不同级别，不会被截断
    user_temp=clear_data[clear_data['user_id']==user_id].sort_values(by=['event_time','attribution_level'],ascending=False)
    # 取所有订单事件，遍历每次订单事件的归因数据
    cb_temp_df=user_temp[user_temp['event_type']=='uc']
    # print(cb_temp_df)
    for row in cb_temp_df.iterrows():
        # 获取订单事件
        cb_series=row[1]
        # 如果通过资源位访问了商品详情，这个时候再记录这个资源位
        # 定义存放查询到的资源位
        find_df=None
        # 定义存放归因等级的顺序和是否归因标记
        # 数据结构={待归因level等级:{'order':顺序, 'is_tag':是否归因标记}}
        level_info={"1":{'order':0, 'is_tag':0},"2":{'order':0, 'is_tag':0},"3":{'order':0, 'is_tag':0}}
        for find_row in user_temp[user_temp['event_time']<cb_series['event_time']].iterrows():
            # 每次找到商品详情页且商品是对应提交的商品，则全部更新标记为1
            if find_row[1]['type']=='relation' and find_row[1]['goods_code']==cb_series['goods_code']:
                for key in level_info:
                    level_info[key]['is_tag']=1
            # 待归因的级别
            attribution_level=find_row[1]['attribution_level']
            # 如果是资源位且该资源位后访问了对应的商品,把资源位存下来，且标记改为0，继续往前找
            if find_row[1]['type']=='attribution' and level_info[attribution_level]['is_tag']==1:
                # 把 series 格式的还转换成 dataframe
                find_temp=find_row[1].to_frame().stack().unstack(0)
                
                find_temp['order']=level_info[attribution_level]['order']
                find_temp['order_code']=cb_series['order_code']
                find_temp['deal_with_amount']=cb_series['deal_with_amount']
                find_temp['order_status']=cb_series['order_status']
                find_temp['risk_audit_ststus']=cb_series['risk_audit_ststus']
                find_temp['order_date']=cb_series['order_date']
                find_temp['pay_date']=cb_series['pay_date']
                
                # 清除待归因级别大于等于此次级别的
                for key in level_info:
                    if attribution_level<=key:
                        level_info[key]['is_tag']=0
                    
                level_info[attribution_level]['order']+=1
                if find_df is None:
                    find_df=find_temp
                else:
                    find_df=pd.concat([find_df,find_temp])
                
                
        # 如果不为空
        if find_df is not None:   
            for attribution_level,items in level_info.items():
                find_df.loc[find_df['attribution_level']==attribution_level,"total"]=items['order']
            # 存数据
            if find_all_df is None:
                find_all_df=find_df
            else:
                find_all_df=pd.concat([find_all_df,find_df])

In [55]:
find_all_df.head()

,id,dt,event_code,event_type,os_type,page_id,page_title,floor_name,item_name,goods_code,...,deal_with_amount,order_status,risk_audit_ststus,order_date,pay_date,properties,type,attribution_level,order,total
2,585888895661903875,2022-06-05,recommend_item_ck,ck,android,1,小象首页,商品tab,底部tab（新）,201883060,...,752900,22,0,2022-06-05,None,"{""goods_name"":""Apple iPad mini 8.3英寸 平板电脑 2021...",attribution,3,0,1.0
4,585888895661903875,2022-06-05,recommend_item_ck,ck,android,1,小象首页,商品tab,底部tab（新）,201883060,...,752900,22,0,2022-06-05,None,"{""goods_name"":""Apple iPad mini 8.3英寸 平板电脑 2021...",attribution,1,0,1.0
1,585888744897781763,2022-06-05,recommend_item_ck,ck,android,1,小象首页,商品tab,底部tab（新）,201936364,...,259900,22,0,2022-06-05,None,"{""goods_name"":""【二手苹果Apple】iPhone 12mini 国行/国际版...",attribution,3,0,1.0
3,585888744897781763,2022-06-05,recommend_item_ck,ck,android,1,小象首页,商品tab,底部tab（新）,201936364,...,259900,22,0,2022-06-05,None,"{""goods_name"":""【二手苹果Apple】iPhone 12mini 国行/国际版...",attribution,1,0,1.0


In [56]:
find_all_df.to_csv('临时结果_{dt}.csv'.format(dt=dt), index=False)

In [57]:
# 防止导入到csv变科学计数法,数仓开发不存在这个问题，不用处理这步
def deal_str(data):
    data = '-'+str(data)
    return data

In [58]:
find_all_df['order_code'] = find_all_df['order_code'].map(deal_str)
find_all_df['event_time'] = find_all_df['event_time'].map(deal_str)
find_all_df['id'] = find_all_df['id'].map(deal_str)

In [59]:
# 导出字段的顺序
colnums_sort=["id","dt","event_code","event_type","os_type","page_id","page_title","floor_name","item_name","event_time","user_id","order_code","goods_code","goods_source","is_recommend","deal_with_amount","risk_audit_ststus","order_status","order_date","pay_date","type","attribution_level","order","total","properties"]

In [60]:
save_df=find_all_df.reindex(columns=colnums_sort)

In [61]:
save_df.to_csv('最终结果数据{dt}.csv'.format(dt=dt), index=False,encoding='utf-8')

## 把清洗的结果保存到hive表

In [62]:
# 最终将结果保存到表里

## 明细数据测试

In [63]:
# all_df[(all_df['user_id']=='16087019') | (all_df['user_id']=='100000000000000')].head()

In [64]:
user_info=clear_data.query('["2503841"] in user_id').sort_values(by=['event_time','attribution_level'],ascending=False)

In [65]:
user_info.to_excel('用户明细x{2503841}a12345.xlsx')

In [66]:
user_info.query('type=="target"')

,id,dt,event_code,event_type,os_type,page_id,page_title,floor_name,item_name,goods_code,...,user_id,order_code,deal_with_amount,order_status,risk_audit_ststus,order_date,pay_date,properties,type,attribution_level


In [67]:
# 这一步，桑题图用的，可以不洗
def clear_title(row):
    # 如果元素不为空，则是资源位，直接用元素名
    # 如果 event_code=page_start，则是详情页，用详情页拼接 goods_code
    # 剩下的则是提交订单，用提交订单拼接 goods_code
    if row['type']=='attribution':
        row['title']="待归因:"+str(row['page_id'])+":"+str(row['page_title'])+":"+str(row['event_code'])+":"+str(row['goods_code'])+":"+str(row['attribution_level'])
    elif row['type']=='relation':
        row['title']='详情页:'+str(row['goods_code'])
    else:
        row['title']='提交订单'+str(row['goods_code'])
    row['title']=row['title']+'@'+str(row['user_id'])
    return row
    return row
user_titile_info=user_info.apply(clear_title,axis='columns')

In [68]:
# user_titile_info['order_code'] = user_titile_info['order_code'].map(deal_str)
# user_titile_info['event_time'] = user_titile_info['event_time'].map(deal_str)

In [69]:
user_titile_info['order_code'] = user_titile_info['order_code'].map(deal_str)

In [70]:
user_titile_info.to_csv('用户明细3{dt}.csv'.format(dt=dt), index=False,encoding='utf-8')

## 对比线上归因逻辑

In [71]:
temp_sql_str='''
select
order_code
,user_id
from dws_xxyp_operation.dws_xxyp_operation_all_attribution_result_data_d_inc_dt
where dt='2022-06-03'
and `order`=0 
and attribution_level=3
and event_code='recommend_item_ck'
and pay_date='2022-05-09'
and split(get_json_object(properties,'$.recommend_source'),'_')[0]='micrpage-v1.0'
and os_type='iOS'
'''

In [72]:
# 执行查询
cursor.execute(temp_sql_str)
# 获取列名
columns = [col[0] for col in cursor.description]

In [73]:
columns

['order_code', 'user_id']

In [74]:
# 查询结果
temp_sql_data=cursor.fetchall()
temp_sql_df=pd.DataFrame(temp_sql_data,columns=columns)

In [75]:
temp_sql_df['order_code'] = temp_sql_df['order_code'].map(deal_str)

In [76]:
temp_sql_df

,order_code,user_id


In [77]:
aaaaaa=find_all_df.query('page_id=="micro_goods_detail" and os_type=="iOS" and order==0 and pay_date=="2022-05-09"')
aaaaaa.to_excel('用户明细x{dt}a12.xlsx')
aaaaaa=aaaaaa[['user_id',"order_code"]]

In [78]:
pd.merge(temp_sql_df,aaaaaa,on='order_code',how='left').to_excel('用户明细x{dt}a123.xlsx')

In [79]:
cursor.close()
conn.close()

In [80]:
str(datetime.datetime.now())

'2022-06-07 13:53:03.062033'